IMPORTING NECESSARY LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import re
import sklearn
import nltk
import bs4
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install beautifulsoup4

IMPORTING THE DATASET

In [ ]:
imdb_data=pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv',names=['review', 'sentiment'],header=0)
print(imdb_data.shape)

VIEWING DATASET

In [ ]:
imdb_data.head(20)

DATA ANALYSIS

In [ ]:
imdb_data.describe()

SENTIMENT COUNT

In [ ]:
imdb_data['sentiment'].value_counts()

In [ ]:
sentiment_values=pd.get_dummies(imdb_data['sentiment'])
sentiment_values=sentiment_values.iloc[:,1].values

DATA CLEANING

In [ ]:
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range (0,len(imdb_data)):
    soup = bs4.BeautifulSoup(imdb_data ['review'][i], "html.parser")
    review=re.sub('\[[^]]*\]',' ',imdb_data['review'][i])
    review=re.sub('[^a-zA-Z]',' ',imdb_data['review'][i])
    review=review.lower()
    review= review. split()
    review=[word for word in review if not word in set(stopwords.words('english'))]
    review = [lemmatizer.lemmatize(word) for word in review]
    review=' '.join (review)
    corpus.append(review)

Term Frequency-Inverse Document Frequency(TFIDF)

In [ ]:
cv = TfidfVectorizer(ngram_range=(1, 3))
tfidf = cv.fit_transform(corpus)

TRAINING AND TESTING DATA

In [ ]:
tfidf_train,tfidf_test,sentiment_values_train,sentiment_values_test=train_test_split(tfidf,sentiment_values,test_size=0.20,random_state=0)

SVC MODEL

In [ ]:
linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_train, sentiment_values_train)

predict = linear_svc.predict(tfidf_test)

CLASSIFICATION REPORT

In [ ]:
report=classification_report(sentiment_values_test, predict,target_names=['Negative','Positive'])
print("Classification Report: \n",report)

CONFUSION MATRIX

In [ ]:
matrix=confusion_matrix(sentiment_values_test, predict)
print("Confusion Matrix: \n",matrix)

ACCURACY

In [ ]:
accuracy=accuracy_score(sentiment_values_test, predict)
print("Accuracy: \n", accuracy)